#  Предсказание стоимости дома

В проекте нам нужно обучить модель линейной регрессии на данных о [жилье в Калифорнии в 1990 году](https://www.kaggle.com/datasets/camnugent/california-housing-prices) (CC0: Public Domain). 

В колонках датасета содержатся следующие данные:
- `longitude` — широта;
- `latitude` — долгота;
- `housing_median_age` — медианный возраст жителей жилого массива;
- `total_rooms` — общее количество комнат в домах жилого массива;
- `total_bedrooms` — общее количество спален в домах жилого массива;
- `population` — количество человек, которые проживают в жилом массиве;
- `households` — количество домовладений в жилом массиве;
- `median_income` — медианный доход жителей жилого массива;
- `median_house_value` — медианная стоимость дома в жилом массиве;
- `ocean_proximity` — близость к океану.

На основе данных нужно предсказать медианную стоимость дома в жилом массиве — `median_house_value`. 

Для оценки качества модели используйте метрики RMSE, MAE и R2.

## Цель исследования

Целью исследования являетя разработка модели прогнозирования стоимости жилья с применением распределенных вычислений. 

### Постановка задачи

В данной работе нам надо, используя распределенные вычисления:
- провести анализ данных;
- построить две модели линейной регрессии, предсказывающие стоимость жилья: одну на полном наборе признаков, другую только на числовых признаках;
- сравнить результаты работы по метрикам RMSE, MAE и R2.

### Основные этапы

1. Инициализируем локальную Spark-сессию.
2. Прочитаем содержимое файла `/datasets/housing.csv`.
3. Выведем типы данных колонок датасета, применив методы pySpark.
4. Выполним предобработку данных:
    - Исследуем данные на наличие пропусков и заполним их.
    - Преобразуем колонку с категориальными значениями техникой One hot encoding.
5. Построим две модели линейной регрессии на разных наборах данных:
    - используя все данные из файла;
    - используя только числовые переменные, исключив категориальные.
6. Для построения модели возьмем оценщик LinearRegression из библиотеки MLlib.
7. Сравним результаты работы линейной регрессии на двух наборах данных по метрикам RMSE, MAE и R2. 
8. Сделаем выводы.

### Выводы

## Загрузка библиотек

In [1]:
from os.path import exists

In [2]:
import pandas as pd 
import numpy as np

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

## Подготовка данных

### Загрузка данных

Определим путь для загрузки данных: в текущей директории или корневой.

In [4]:
data_path = './datasets/housing.csv'
if not exists(data_path):
    data_path = data_path[1:]
data_path

'./datasets/housing.csv'

Инициализируем локальную Spark-сессию.

In [5]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("California Housing median value prediction") \
                    .getOrCreate()

22/08/02 16:26:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Прочитаем содержимое файла данных.

In [23]:
df_housing = spark.read.load(data_path, 
                             format='csv',
                             sep=',',
                             inferSchema=True,
                             header='true'
                            )
df_housing.count() 

20640

Выведем типы данных колонок датасета.

In [24]:
df_housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



**ВЫВОД**

Мы загрузили датасет в котором 20640 объектов и 10 признаков. 

9 признаков из 10 имеют тип double, один признак строковый. 

Все признаки могут содержать пустые значения.

### Предобработка данных

Посмотрим на первые 10 строк данных. 

In [53]:
df_housing.limit(10).toPandas().T

,0,1,2,3,4,5,6,7,8,9
ocean_proximity,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY
longitude,-122.23,-122.22,-122.24,-122.25,-122.25,-122.25,-122.25,-122.25,-122.26,-122.25
latitude,37.88,37.86,37.85,37.85,37.85,37.85,37.84,37.84,37.84,37.84
housing_median_age,41.0,21.0,52.0,52.0,52.0,52.0,52.0,52.0,42.0,52.0
total_rooms,880.0,7099.0,1467.0,1274.0,1627.0,919.0,2535.0,3104.0,2555.0,3549.0
total_bedrooms,129.0,1106.0,190.0,235.0,280.0,213.0,489.0,687.0,665.0,707.0
population,322.0,2401.0,496.0,558.0,565.0,413.0,1094.0,1157.0,1206.0,1551.0
households,126.0,1138.0,177.0,219.0,259.0,193.0,514.0,647.0,595.0,714.0
median_income,8.3252,8.3014,7.2574,5.6431,3.8462,4.0368,3.6591,3.12,2.0804,3.6912
median_house_value,452600.0,358500.0,352100.0,341300.0,342200.0,269700.0,299200.0,241400.0,226700.0,261100.0


In [56]:
df_housing.describe().toPandas().set_index('summary').T

summary,count,mean,stddev,min,max
ocean_proximity,20640,None,None,<1H OCEAN,NEAR OCEAN
longitude,20640,-119.56970445736148,2.003531723502584,-124.35,-114.31
latitude,20640,35.6318614341087,2.135952397457101,32.54,41.95
housing_median_age,20640,28.639486434108527,12.58555761211163,1.0,52.0
total_rooms,20640,2635.7630813953488,2181.6152515827944,2.0,39320.0
total_bedrooms,20640,537.8808920871179,419.2677350499039,1.0,6445.0
population,20640,1425.4767441860465,1132.46212176534,3.0,35682.0
households,20640,499.5396802325581,382.3297528316098,1.0,6082.0
median_income,20640,3.8706710029070246,1.899821717945263,0.4999,15.0001
median_house_value,20640,206855.81690891474,115395.61587441359,14999.0,500001.0


Мы видим, что признак `ocean_proximity` является категориальным. Признаки `longitude` и `longitude` являются географическими координатами и из соображений здарвого смысла должны быть связаны с признаком `ocean_proximity`, описывая географическое положение жилового массива.

Остальные признаки являются количественными. Сохраним названия количественных и категориальных признаков.

In [60]:
numerical_columns = df_housing.columns[1:]
categorical_columns = df_housing.columns[:1]

In [61]:
numerical_columns

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value']

In [62]:
categorical_columns

['ocean_proximity']

#### Анализ пропусков

Исследуем данные на наличие пропусков и заполним их.

In [25]:
na_count = df_housing \
           .select([F.count(F.when(F.col(c).contains('None') | \
                                   F.col(c).contains('NULL') | \
                                   (F.col(c) == '' ) | \
                                   F.col(c).isNull() | \
                                   F.isnan(c), c 
                                  )).alias(c)
                    for c in df_housing.columns])

na_count.toPandas().T

,0
longitude,0
latitude,0
housing_median_age,0
total_rooms,0
total_bedrooms,207
population,0
households,0
median_income,0
median_house_value,0
ocean_proximity,0


Признак `total_bedrooms` содержит 207 пропусков. Посмотрим на примеры строк с пропусками.

In [26]:
df_housing.filter(df_housing.total_bedrooms.isNull()).toPandas().head(12).T

,0,1,2,3,4,5,6,7,8,9,10,11
longitude,-122.16,-122.17,-122.28,-122.24,-122.1,-122.14,-121.77,-121.95,-121.98,-122.01,-122.08,-119.75
latitude,37.77,37.75,37.78,37.75,37.69,37.67,39.66,38.03,37.96,37.94,37.88,36.71
housing_median_age,47.0,38.0,29.0,45.0,41.0,37.0,20.0,5.0,22.0,23.0,26.0,38.0
total_rooms,1256.0,992.0,5154.0,891.0,746.0,3342.0,3759.0,5526.0,2987.0,3741.0,2947.0,1481.0
total_bedrooms,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
population,570.0,732.0,3741.0,384.0,387.0,1635.0,1705.0,3207.0,1420.0,1339.0,825.0,1543.0
households,218.0,259.0,1273.0,146.0,161.0,557.0,600.0,1012.0,540.0,499.0,626.0,372.0
median_income,4.375,1.6196,2.5762,4.9489,3.9063,4.7933,4.712,4.0767,3.65,6.7061,2.933,1.4577
median_house_value,161900.0,85100.0,173400.0,247100.0,178400.0,186900.0,158600.0,143100.0,204100.0,322300.0,85000.0,49800.0
ocean_proximity,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,INLAND,INLAND,INLAND,NEAR BAY,NEAR BAY,INLAND


Мы видим, что пропуски есть у объектах, имеющих разные значения удаленности от океана. Возможно типы строений для разной удаленности от океана отличаются. Тогда мы можем заполнить пропуски средними значениями для соответвующей категории дальности. 

Проверим эту гипотезу. Посчитаем среднее значение спален по значениям удаленности от океана.

In [31]:
bedrooms_by_ocean_proximity = [{ r['ocean_proximity']: 
                                 r['avg_bedrroms'] }  
                               for r in df_housing \
                                        .select('ocean_proximity', 'total_bedrooms') \
                                        .groupBy('ocean_proximity') \
                                        .avg('total_bedrooms') \
                                        .toDF('ocean_proximity', 'avg_bedrroms') \
                                        .collect() ]

bedrooms_by_ocean_proximity

[{'ISLAND': 420.4},
 {'NEAR OCEAN': 538.6156773211568},
 {'NEAR BAY': 514.1828193832599},
 {'<1H OCEAN': 546.5391852999778},
 {'INLAND': 533.8816194581281}]

Мы видим, что среднее значение количества спален зависит от дальности жилого массива от океана. Заполним пропуски средними значениями в зависимости от значения `ocean_proximity`.

Сначала создадим DataFrame со средними значениями.

In [43]:
bedrooms_avg = df_housing \
               .select('ocean_proximity', 'total_bedrooms') \
               .groupBy('ocean_proximity') \
               .agg(F.mean('total_bedrooms').alias('total_bedrooms_avg'))

Потом присоединим средние сзначения к основному датасету слева.

In [44]:
df_extended = df_housing.join(bedrooms_avg, ['ocean_proximity'], 'left')

In [46]:
df_extended.select('ocean_proximity', 'total_bedrooms', 'total_bedrooms_avg').show(5)

+---------------+--------------+------------------+
|ocean_proximity|total_bedrooms|total_bedrooms_avg|
+---------------+--------------+------------------+
|       NEAR BAY|         129.0| 514.1828193832599|
|       NEAR BAY|        1106.0| 514.1828193832599|
|       NEAR BAY|         190.0| 514.1828193832599|
|       NEAR BAY|         235.0| 514.1828193832599|
|       NEAR BAY|         280.0| 514.1828193832599|
+---------------+--------------+------------------+
only showing top 5 rows



Теперь заменим пустые значения в столбце total_bedrooms на средние значения, применив трансформацию `coalesce`, которая сохранить непустое значение без изменений, а пустые заменит значениями из переданного столбца.

In [48]:
df_housing = df_extended \
             .withColumn('total_bedrooms', F.coalesce('total_bedrooms', 'total_bedrooms_avg')) \
             .drop('total_bedrooms_avg')

Проверим, что непустые значения сохранились.

In [50]:
df_housing.select('ocean_proximity', 'total_bedrooms').show(5)

+---------------+--------------+
|ocean_proximity|total_bedrooms|
+---------------+--------------+
|       NEAR BAY|         129.0|
|       NEAR BAY|        1106.0|
|       NEAR BAY|         190.0|
|       NEAR BAY|         235.0|
|       NEAR BAY|         280.0|
+---------------+--------------+
only showing top 5 rows



Все в порядке. Проверим, что пропуски заполнены.

In [51]:
na_count = df_housing \
           .select([F.count(F.when(F.col(c).contains('None') | \
                                   F.col(c).contains('NULL') | \
                                   (F.col(c) == '' ) | \
                                   F.col(c).isNull() | \
                                   F.isnan(c), c 
                                  )).alias(c)
                    for c in df_housing.columns])

na_count.toPandas().T

,0
ocean_proximity,0
longitude,0
latitude,0
housing_median_age,0
total_rooms,0
total_bedrooms,0
population,0
households,0
median_income,0
median_house_value,0


Пропусков больше нет. И проверим, что средние значения по категориям дальности не изменились, поскольку мы заполнили пропуски средними же значениями.

In [52]:
bedrooms_by_ocean_proximity = [{ r['ocean_proximity']: 
                                 r['avg_bedrroms'] }  
                               for r in df_housing \
                                        .select('ocean_proximity', 'total_bedrooms') \
                                        .groupBy('ocean_proximity') \
                                        .avg('total_bedrooms') \
                                        .toDF('ocean_proximity', 'avg_bedrroms') \
                                        .collect() ]

bedrooms_by_ocean_proximity

[{'ISLAND': 420.4},
 {'NEAR OCEAN': 538.6156773211568},
 {'NEAR BAY': 514.1828193832598},
 {'<1H OCEAN': 546.5391852999772},
 {'INLAND': 533.8816194581278}]

Все в порядке. 

**ВЫВОД**

Мы обнаружили 207 пропущенных значений признака `total_bedrooms` и заменили их средними значениями по категориям удаленности от океана.

#### Кодирование категориальных признаков

Преобразуем колонку с категориальными значениями техникой One hot encoding.

### Разделение  на обучающую и тестовые выборки

## Обучение моделей

Для построения модели возьмем оценщик LinearRegression из библиотеки MLlib. 

Построим две модели линейной регрессии на разных наборах данных: полном наборе и только на числовых признаках.

### Полный набор данных

### Только числовые признаки

## Анализ результатов

Сравним результаты работы двух моделей на тестовой выборке по метрикам RMSE, MAE и R2. 

## Выводы